In [182]:
import numpy as np 
import pandas as pd
from plotly import graph_objects as go
import plotly.express as px
import calendar

In [183]:
home = pd.read_csv('https://github.com/jerometanel/Ecommerce-Funnel-EDA/blob/main/home_page_table.csv?raw=true')
search = pd.read_csv('https://github.com/jerometanel/Ecommerce-Funnel-EDA/blob/main/search_page_table.csv?raw=true')
payment = pd.read_csv('https://github.com/jerometanel/Ecommerce-Funnel-EDA/blob/main/payment_page_table.csv?raw=true')
confirmation = pd.read_csv('https://github.com/jerometanel/Ecommerce-Funnel-EDA/blob/main/payment_confirmation_table.csv?raw=true')
user = pd.read_csv('https://github.com/jerometanel/Ecommerce-Funnel-EDA/blob/main/user_table.csv?raw=true')

## **Count of Visitors on Each Page**

In [184]:
# count user_id of each page to display the sum of visitors to each page

page_count = pd.DataFrame([['Home', home['user_id'].count()],
                            ['Search', search['user_id'].count()],
                            ['Payment', payment['user_id'].count()],
                            ['Confirmation', confirmation['user_id'].count()]],
                            columns = ['Step', 'Count'])

page_count

,Step,Count
0,Home,90400
1,Search,45200
2,Payment,6030
3,Confirmation,452


## **Visualising Count of Visitors by Funnel Analysis**

In [185]:
# visualising funnel with graph_objects

fig = go.Figure(go.Funnel(
    y = ['Home', 'Search', 'Payment', 'Confirmation'],
    x = [90400, 45200, 6030, 425],
    textposition = 'outside',
    textinfo = 'value + percent initial'))

fig.show()

## **Merge Tables**

In [186]:
# renaming columns in original datasets 

home = home.rename(columns = {'home_page':'Home'})
search = search.rename(columns = {'search_page':'Search'})
payment = payment.rename(columns = {'payment_page':'Payment'})
confirmation = confirmation.rename(columns = {'payment_confirmation_page':'Confirmation'})


In [187]:
# merge all the tables

flow = user.merge(home, how = 'outer', on = 'user_id').merge(search, 
                        how = 'outer', on = 'user_id').merge(payment, 
                        how = 'outer', on = 'user_id').merge(confirmation, 
                        how = 'outer', on = 'user_id')

flow.head()

,user_id,date,device,sex,Home,Search,Payment,Confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [188]:
# check info of dataset

flow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90400 entries, 0 to 90399
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       90400 non-null  int64 
 1   date          90400 non-null  object
 2   device        90400 non-null  object
 3   sex           90400 non-null  object
 4   Home          90400 non-null  object
 5   Search        45200 non-null  object
 6   Payment       6030 non-null   object
 7   Confirmation  452 non-null    object
dtypes: int64(1), object(7)
memory usage: 6.2+ MB


In [189]:
# check number of unique user_id

users = flow['user_id'].nunique()
users

90400

## **Discover Correlations**

**Transforming categorical features**

In [190]:
# one hot encoding on all features : flow2

flow2 = pd.get_dummies(data=flow, 
                        columns = ['device', 'sex', 'Home', 'Search', 'Payment', 'Confirmation'],
                        dummy_na = False)
    
flow2.head()

,user_id,date,device_Desktop,device_Mobile,sex_Female,sex_Male,Home_home_page,Search_search_page,Payment_payment_page,Confirmation_payment_confirmation_page
0,450007,2015-02-28,1,0,1,0,1,0,0,0
1,756838,2015-01-13,1,0,0,1,1,0,0,0
2,568983,2015-04-09,1,0,0,1,1,1,0,0
3,190794,2015-02-18,1,0,1,0,1,1,0,0
4,537909,2015-01-15,1,0,0,1,1,0,0,0


In [191]:
# rename column names

flow2.rename(columns={'Home_home_page' : 'Home',
                        'Search_search_page' : 'Search',
                        'Payment_payment_page' : 'Payment',
                        'Confirmation_payment_confirmation_page' : 'Confirmation',
                        'device_Desktop' : 'Desktop',
                        'device_Mobile' : 'Mobile',
                        'sex_Female' : 'Female',
                        'sex_Male' : 'Male'}, inplace = True)

flow2.head()

,user_id,date,Desktop,Mobile,Female,Male,Home,Search,Payment,Confirmation
0,450007,2015-02-28,1,0,1,0,1,0,0,0
1,756838,2015-01-13,1,0,0,1,1,0,0,0
2,568983,2015-04-09,1,0,0,1,1,1,0,0
3,190794,2015-02-18,1,0,1,0,1,1,0,0
4,537909,2015-01-15,1,0,0,1,1,0,0,0


In [192]:
# overview of correlation

flow2.corr().style.background_gradient(cmap='coolwarm')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/io/formats/style.py:1126: RuntimeWarning:

All-NaN slice encountered

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/io/formats/style.py:1127: RuntimeWarning:

All-NaN slice encountered



,user_id,Desktop,Mobile,Female,Male,Home,Search,Payment,Confirmation
user_id,1.000000,-0.001100,0.001100,-0.000987,0.000987,nan,-0.000991,-0.003305,-0.005167
Desktop,-0.001100,1.000000,-1.000000,-0.000927,0.000927,nan,-0.000000,-0.094519,-0.050209
Mobile,0.001100,-1.000000,1.000000,0.000927,-0.000927,nan,0.000000,0.094519,0.050209
Female,-0.000987,-0.000927,0.000927,1.000000,-1.000000,nan,0.006128,0.008276,0.004901
Male,0.000987,0.000927,-0.000927,-1.000000,1.000000,nan,-0.006128,-0.008276,-0.004901
Home,nan,nan,nan,nan,nan,nan,nan,nan,nan
Search,-0.000991,-0.000000,0.000000,0.006128,-0.006128,nan,1.000000,0.267340,0.070888
Payment,-0.003305,-0.094519,0.094519,0.008276,-0.008276,nan,0.267340,1.000000,0.265161
Confirmation,-0.005167,-0.050209,0.050209,0.004901,-0.004901,nan,0.070888,0.265161,1.000000


## **Create dataset for powerbi (aditional activity)**

In [193]:
flow2['Sex'] = flow2['Male']
flow2.drop(['Female', 'Male', 'Mobile'], axis=1, inplace=True)

In [227]:
# create month column
flow2['month'] = pd.DatetimeIndex(flow2['date']).month
flow2['month'] = flow2['month'].apply(lambda x: calendar.month_abbr[x])

flow2.head()

,user_id,date,Desktop,Home,Search,Payment,Confirmation,Sex,month
0,450007,2015-02-28,1,1,0,0,0,0,Feb
1,756838,2015-01-13,1,1,0,0,0,1,Jan
2,568983,2015-04-09,1,1,1,0,0,1,Apr
3,190794,2015-02-18,1,1,1,0,0,0,Feb
4,537909,2015-01-15,1,1,0,0,0,1,Jan


In [228]:
flow2.to_csv('funnel2.csv', index=False)

## **Comparing Visits by Gender**

In [196]:
gender = flow.groupby(['sex']).user_id.count().reset_index()

gender

,sex,user_id
0,Female,45075
1,Male,45325


## **Comparing Visits per Page by Gender**

In [197]:
# create pivot table function for gender and page

def gender_pivot(page):
    gender_page = flow.groupby(['sex', page]).user_id.count().reset_index()
    gender_pivot = gender_page.pivot(index='sex', columns = page, values = 'user_id')
   
    return gender_pivot

In [198]:
# home pivot using the function created

home_pivot = gender_pivot('Home')
home_pivot

Home,home_page
sex,
Female,45075
Male,45325


In [199]:
# repeat for search, payment, confirmation

search_pivot = gender_pivot('Search')
payment_pivot = gender_pivot('Payment')
confirmation_pivot = gender_pivot('Confirmation')

In [200]:
# merge pivot tables and exclude duplicated sex

gender_drop = home_pivot.merge(search_pivot, 
                                how = 'outer', on = 'sex').merge(payment_pivot, 
                                how = 'outer', on = 'sex').merge(confirmation_pivot, 
                                how = 'outer', on = 'sex')
    
gender_drop

,home_page,search_page,payment_page,payment_confirmation_page
sex,,,,
Female,45075,22676,3100,241
Male,45325,22524,2930,211


In [201]:
# using graph_objects to plot the above observations

fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Female',
    y = ['Home', 'Search', 'Payment', 'Confirmation'],
    x = [45075, 22676, 3100, 241],
    textposition = 'inside',
    textinfo = 'value + percent initial'))

fig.add_trace(go.Funnel(
    name = 'Male',
    y = ['Home', 'Search', 'Payment', 'Confirmation'],
    x = [45325, 22524, 2930, 211],
    textposition = 'inside',
    textinfo = 'value + percent previous'))

fig.show()

## **Comparing Visits by Device**

In [202]:
device_used = flow.groupby('device').user_id.count().reset_index()

device_used

,device,user_id
0,Desktop,60200
1,Mobile,30200


## **Comparing Visits per Page by Device**

In [203]:
# create pivot table function for device and page

def device_pivot(page):
    device_page = flow.groupby(['device', page]).user_id.count().reset_index()
    device_pivot = device_page.pivot(index='device', columns = page, values = 'user_id')
   
    return device_pivot

In [204]:
# use pivot function for all pages
home_pivot = device_pivot('Home')
search_pivot = device_pivot('Search')
payment_pivot = device_pivot('Payment')
confirmation_pivot = device_pivot('Confirmation')

home_pivot

Home,home_page
device,
Desktop,60200
Mobile,30200


In [205]:
# merge pivot tables and exclude duplicated device

device_drop = home_pivot.merge(search_pivot, 
                                how = 'outer', on = 'device').merge(payment_pivot, 
                                how = 'outer', on = 'device').merge(confirmation_pivot, 
                                how = 'outer', on = 'device')
    
device_drop

,home_page,search_page,payment_page,payment_confirmation_page
device,,,,
Desktop,60200,30100,3010,150
Mobile,30200,15100,3020,302


In [206]:
# using graph_objects to plot the above observations

fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Desktop',
    y = ['Home', 'Search', 'Payment', 'Confirmation'],
    x = [60200, 30100, 3010, 150],
    textposition = 'inside',
    textinfo = 'value + percent initial'))

fig.add_trace(go.Funnel(
    name = 'Mobile',
    y = ['Home', 'Search', 'Payment', 'Confirmation'],
    x = [30200, 15100, 302, 302],
    textposition = 'inside',
    textinfo = 'value + percent previous'))

fig.show()

## **Comparing Visits Base on Gender & Device**

In [207]:
# pivot table for device and sex

flow_device = flow.groupby(['sex', 'device']).user_id.count().reset_index()
flow_device_pivot = flow_device.pivot(index = 'sex', columns = 'device', values ='user_id')

flow_device_pivot

device,Desktop,Mobile
sex,,
Female,29997,15078
Male,30203,15122


## **Comparing Visits per Per Page base on Gender & Device**

In [208]:
# create function for pivot table 

def pages_pivot(page):
    pages_pivot = flow.groupby(['sex', 'device', page]).user_id.count().reset_index()
    return pages_pivot


In [209]:
pages_pivot('Home')

,sex,device,Home,user_id
0,Female,Desktop,home_page,29997
1,Female,Mobile,home_page,15078
2,Male,Desktop,home_page,30203
3,Male,Mobile,home_page,15122


In [210]:
pages_pivot('Search')

,sex,device,Search,user_id
0,Female,Desktop,search_page,15091
1,Female,Mobile,search_page,7585
2,Male,Desktop,search_page,15009
3,Male,Mobile,search_page,7515


In [211]:
pages_pivot('Payment')

,sex,device,Payment,user_id
0,Female,Desktop,payment_page,1530
1,Female,Mobile,payment_page,1570
2,Male,Desktop,payment_page,1480
3,Male,Mobile,payment_page,1450


In [212]:
pages_pivot('Confirmation')

,sex,device,Confirmation,user_id
0,Female,Desktop,payment_confirmation_page,74
1,Female,Mobile,payment_confirmation_page,167
2,Male,Desktop,payment_confirmation_page,76
3,Male,Mobile,payment_confirmation_page,135


In [213]:
# create dataframe table : device_gender_sales

data = {'Stage' : ['Home', 'Search', 'Payment', ' Confirmation'],
        'Desktop Male' : [30203, 15009, 1480, 76],
        'Mobile Male' : [15122, 7515, 1450, 135], 
        'Desktop Female':[29997, 15091, 1530, 74], 
        'Mobile Female':[15078, 7585, 1570, 167]}

device_gender_sales = pd.DataFrame(data)

device_gender_sales.T

,0,1,2,3
Stage,Home,Search,Payment,Confirmation
Desktop Male,30203,15009,1480,76
Mobile Male,15122,7515,1450,135
Desktop Female,29997,15091,1530,74
Mobile Female,15078,7585,1570,167


In [214]:
# plot the funnel 

fig = px.funnel(device_gender_sales, x = ['Desktop Male', 'Mobile Male', 
                                            'Desktop Female', 'Mobile Female'],
                                            y = 'Stage')

fig.show()

## **Customer Churn**

In [215]:
flow.head()

,user_id,date,device,sex,Home,Search,Payment,Confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [216]:
# create month column

flow['month'] = pd.DatetimeIndex(flow['date']).month

flow.head()

,user_id,date,device,sex,Home,Search,Payment,Confirmation,month
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN,2
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN,1
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN,4
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN,2
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN,1


In [217]:
# change numerical month to name of month

flow['month'] = flow['month'].apply(lambda x: calendar.month_abbr[x])

flow.head()

,user_id,date,device,sex,Home,Search,Payment,Confirmation,month
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN,Feb
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN,Jan
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN,Apr
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN,Feb
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN,Jan


## **Churn Rate Across All Pages**

In [218]:
# observe how churn occur in each month 
# function to create pivot table 

def month_page_pivot(page):
    month_page_dist = flow.groupby(['month', page]).user_id.count().reset_index()
    month_page_pivot = month_page_dist.pivot(index = page, columns = 'month', values = 'user_id')
    return month_page_pivot

In [219]:
# applying above function to each page

home_pivot = month_page_pivot('Home')
search_pivot = month_page_pivot('Search')
payment_pivot = month_page_pivot('Payment')
confirmation_pivot = month_page_pivot('Confirmation')

In [220]:
# merging the pivot tables

month_drop_dist = pd.concat([home_pivot, search_pivot, payment_pivot, confirmation_pivot], axis = 0)
month_drop_dist.T

,home_page,search_page,payment_page,payment_confirmation_page
month,,,,
Apr,22600,9080,597,46
Feb,22600,13687,2412,173
Jan,22600,13554,2390,189
Mar,22600,8879,631,44


In [221]:
# observe how churn occur according to devices in individual months

def month_device_page(page):
    month_device_page = flow.groupby(['month', 'device', page]).user_id.count().reset_index()
    return month_device_page

In [222]:
month_device_page('Home')

,month,device,Home,user_id
0,Apr,Desktop,home_page,15050
1,Apr,Mobile,home_page,7550
2,Feb,Desktop,home_page,15050
3,Feb,Mobile,home_page,7550
4,Jan,Desktop,home_page,15050
5,Jan,Mobile,home_page,7550
6,Mar,Desktop,home_page,15050
7,Mar,Mobile,home_page,7550


In [223]:
month_device_page('Search')

,month,device,Search,user_id
0,Apr,Desktop,search_page,7569
1,Apr,Mobile,search_page,1511
2,Feb,Desktop,search_page,7632
3,Feb,Mobile,search_page,6055
4,Jan,Desktop,search_page,7529
5,Jan,Mobile,search_page,6025
6,Mar,Desktop,search_page,7370
7,Mar,Mobile,search_page,1509


In [224]:
month_device_page('Payment')

,month,device,Payment,user_id
0,Apr,Desktop,payment_page,302
1,Apr,Mobile,payment_page,295
2,Feb,Desktop,payment_page,1191
3,Feb,Mobile,payment_page,1221
4,Jan,Desktop,payment_page,1221
5,Jan,Mobile,payment_page,1169
6,Mar,Desktop,payment_page,296
7,Mar,Mobile,payment_page,335


In [225]:
month_device_page('Confirmation')

,month,device,Confirmation,user_id
0,Apr,Desktop,payment_confirmation_page,22
1,Apr,Mobile,payment_confirmation_page,24
2,Feb,Desktop,payment_confirmation_page,54
3,Feb,Mobile,payment_confirmation_page,119
4,Jan,Desktop,payment_confirmation_page,60
5,Jan,Mobile,payment_confirmation_page,129
6,Mar,Desktop,payment_confirmation_page,14
7,Mar,Mobile,payment_confirmation_page,30


In [226]:
flow.to_csv('funnel.csv', index=False)